In [ ]:
from PIL import Image
import numpy
import random
import math

apple = Image.open("banner.png")
array = numpy.array(apple)


for i in range(len(array)):
    for j in range(len(array[i])):
        if array[i][j][0] > 150:
            array[i][j] = [0,min(255,2 * array[i][j][1]),min(255,2 * array[i][j][2]), 255]
        array[i][j][0] += random.uniform(-0.1, 0.1)
        array[i][j][1] += random.uniform(-0.1, 0.1)
        array[i][j][2] += random.uniform(-0.1, 0.1)
        array[i][j][3] += random.uniform(-0.1, 0.1)

apple2 = Image.fromarray(array)
apple2.show()

In [2]:
from PIL import Image
import numpy
from Controller import converter
import cv2

leftImg, rightImg = numpy.split(numpy.array(Image.open("img1.png")), 2, axis=1)

def show(img):
    Image.fromarray(img).show()

filteredLeftImg = converter.filter(leftImg)
leftEdgesImage, gLeftIn, rLeftIn = cv2.split(filteredLeftImg)

show(leftEdgesImage)

filteredRightImg = converter.filter(rightImg)
rightEdgesImage, gRightIn, rRightIn = cv2.split(filteredRightImg)

distances, heights = converter.getDistances(leftEdgesImage, rightEdgesImage)

# image = []

print(heights)
print(distances)

for i in distances:
    print(i)
    # image.append([distances[i][2], distances[i][2], distances[i][2]])
# show(image)
# show(numpy.concatenate(([distances], [distances], [distances]), axis=0))
rBlobs, gBlobs = converter.getBlobs(rLeftIn, gLeftIn, rRightIn, gRightIn)

outerWallLandmarks, innerWallLandmarks, rBlobLandmarks, gBlobLandmarks = converter.getLandmarks(distances, rBlobs, gBlobs)

print(outerWallLandmarks, innerWallLandmarks, rBlobLandmarks, gBlobLandmarks)

[[ 4.92650981e+02  4.98743309e+02  6.18988150e+02  7.91543298e-01]
 [ 4.93544101e+02  5.00544101e+02  4.90544101e+01  7.92439676e-01]
 [ 4.95048459e+02  5.03569834e+02 -8.04328456e+02  7.93931121e-01]
 [-3.00000000e+00  1.00000000e+01  1.00000000e+04  1.86225312e+00]
 [-4.95048459e+02  5.03569834e+02 -8.04328456e+02  2.34766153e+00]
 [-4.92650981e+02  4.98743309e+02  6.18988150e+02  2.35004936e+00]]
[492.6509805  498.74330862 618.98814957   0.7915433 ]
[493.54410082 500.54410082  49.05441008   0.79243968]
[ 4.95048459e+02  5.03569834e+02 -8.04328456e+02  7.93931121e-01]
[-3.00000000e+00  1.00000000e+01  1.00000000e+04  1.86225312e+00]
[-495.04845887  503.56983373 -804.32845557    2.34766153]
[-492.6509805   498.74330862  618.98814957    2.35004936]
[] [array([ 4.95048459e+02,  5.03569834e+02, -8.04328456e+02,  7.93931121e-01]), array([-495.04845887,  503.56983373, -804.32845557,    2.34766153])] [] []


In [ ]:
import math

horizontalFov = 155
verticalFov = 115
imageWidth = 272
imageHeight = 154
focalLength = ((imageHeight / 2) / math.tan(math.pi * (verticalFov / 2) / 180))
wallHeight = 10
centerOffset = 10
dist = wallHeight * math.sqrt(focalLength**2 + (imageWidth / 2 - 29)**2) / 36
# return (dir * (3 + a[1] * dist), (10 + a[2] * dist), dist)

print(focalLength)
dist